In [ ]:
import pandas as pd

import matplotlib.pyplot as plt

import pandas as pd
import numpy as np
import sklearn
import tensorflow as tf 
import matplotlib.gridspec as gridspec
import math
from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.utils import shuffle
import datetime
from sklearn.metrics import confusion_matrix

In [ ]:
data = pd.read_csv("creditcard.csv")
data.info()

In [ ]:
data.shape

In [ ]:
import matplotlib.pyplot as plt
data["Class"].value_counts().plot(kind='bar')
plt.show()

In [ ]:
# Create Column for non-fraud transaction
data.loc[data.Class == 0, 'normal'] = 1
data.loc[data.Class == 1, 'normal'] = 0

# Rename the Class to fraud

data = data.rename(columns ={'Class' : 'fraud'})


In [ ]:
from sklearn.preprocessing import StandardScaler
data['normAmount'] = StandardScaler().fit_transform(data['Amount'].values.reshape(-1,1))

In [ ]:
data = data.drop(['Time', 'Amount'], axis=1)

In [ ]:
fraud = data[data.fraud == 1]
normal = data[data.normal ==1]
fraud.shape
normal.shape

In [ ]:
from sklearn.utils import resample
normal =resample(normal, replace=False, #sample without replacement
                         n_samples =492, # to match with minority class
                         random_state =123 #reproducible results
                )
data

In [ ]:
# Set X_train to 80% of fraduelent transactions
X_train = fraud.sample(frac =0.8)
count_frauds =len(X_train)
# Add 80% of normal tansactions to X_train
X_train = pd.concat([X_train, normal.sample(frac=0.8)], axis =0)
# X_test now contains all the transactions not in X_train
X_test = data.loc[~data.index.isin(X_train.index)]

In [ ]:
from sklearn.utils import shuffle

# shuffle the dataframe so that training can be done in a random fashion
X_train = shuffle(X_train)
X_test = shuffle(X_test)
print(X_train)

In [ ]:
#Adding our target features to y_train and y_test
y_train = X_train.fraud
y_train = pd.concat([y_train, X_train.normal], axis =1)

y_test = X_test.fraud
y_test = pd.concat([y_test, X_test.normal], axis =1)

In [ ]:
# Dropping target features from X_train & X_test
X_train = X_train.drop(['fraud', 'normal'], axis =1)
X_test = X_test.drop(['fraud', 'normal'], axis =1)

In [ ]:
X_train.shape
# getting required i/p & o/p matrices
x_input_train = X_train.as_matrix()
y_input_train = y_train.as_matrix()

x_input_test = X_test.as_matrix()
y_input_test = y_test.as_matrix()

X_test.shape
y_train.shape
y_test.shape

In [ ]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

# Defining model hyperparameters
learning_rate = 0.01
batch_size =100

#placeholders
x =tf.placeholder(tf.float32,[None,29], name= 'x_placeholder')
y =tf.placeholder(tf.float32,[None,2], name= 'y_placeholder')

num_classes = 2 
feature_num = 29
#hidden laye sizes
hidden1_unit = 29
hidden2_unit = 10
hidden3_unit = 10
hidden4_unit = 10


In [ ]:
import math 
# Model layers
weights1= tf.Variable(tf.truncated_normal([29, hidden1_unit], stddev=1.0/math.sqrt(float(feature_num))), name = 'weights')
biases1 = tf.Variable(tf.zeros(hidden1_unit), name='biases')
hidden1 = tf.nn.sigmoid(tf.matmul(x,weights1)*biases1)
hidden1 = tf.nn.dropout(hidden1, 0.5)

weights2= tf.Variable(tf.truncated_normal([hidden1_unit, hidden2_unit], stddev=1.0/math.sqrt(float(hidden1_unit))), name = 'weights')
biases2 = tf.Variable(tf.zeros(hidden2_unit), name='biases')
hidden2 = tf.nn.sigmoid(tf.matmul((hidden1_unit),(weights2))*biases2)
hidden2 = tf.nn.dropout(hidden2, 0.5)

weights3= tf.Variable(tf.truncated_normal([hidden2_unit, hidden3_unit], stddev=1.0/math.sqrt(float(hidden2_unit))), name = 'weights')
biases3 = tf.Variable(tf.zeros(hidden3_unit), name='biases')
hidden3 = tf.nn.sigmoid(tf.matmul((hidden2_unit),(weights3))*biases3)
hidden3 = tf.nn.dropout(hidden3, 0.5)

#Linear

weights4= tf.Variable(tf.truncated_normal([hidden3_unit, num_classes], stddev=1.0/math.sqrt(float(hidden5_unit))), name = 'weights')
biases4 = tf.Variable(tf.zeros(num_classes), name='biases')
logits = tf.nn.softmax(tf.matmul(hidden3, weights5) +biases5)

loss =tf.cast((tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits =logits, labels=Y)) +
                                                                      0.01*tf.nn.l2_loss(weights1)+
                                                                      0.01*tf.nn.l2_loss(biases)+
                                                                      0.01*tf.nn.l2_loss(weights4)+
                                                                      0.01*tf.nn.l2_loss(biases)), tf.float32)
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss)

correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(Y,1))

accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
train_predictions = tf.nn.softmax(logits)

In [ ]:
def next_batch(num, data, labels):
    # return a total no of 'num' random samples with labels
    idx = np.arange(0, len(data))
    np.random.shuffle(idx)
    idx =idx[:num]
    data_shuffle =[data[i] for i in idx]
    labels_shuffle =[labels[i] for i in idx]
    
    return np.asarray(data_shuffle), np.asarray(labels_shuffle)

In [ ]:
#session
import datetime
sess = tf.InteractiveSession()
tf.summary.scalar('accuracy', accuracy)
time_string = datetime.datetime.now().isoformat()
merged = tf.summary.merge_all()
train_writer = tf.summary.FileWriter(f'./train', sess.graph)
test_writer = tf.summary.FileWriter(f'./test', sess.graph)

tf.global_variables_initializer().run()

In [ ]:
# Tensorflow Session 

print("Initialized")
a=[]
b=[]
c=[]
d=[]

for step in range(2000): #epochs
    #defining offset
    offset = (step*batch_size)%(y_input_train.shape[0] - batch_size)
    #creating batch data and labels
    batch_data =x_input_train[offset:(offset + batch_size)]
    batch_labels = y_input_train[offset:(offset + batch_size)]
    
    batch_x, batch_y = batch_data, batch_labels
    sess.run(optimizer, feed_dict= {X: batch_x, Y: batch_y})
    sess.run(accuracy, feed_dict= {X: x_input_test, Y: y_input_test}
    
    if step%50 ==0:
             train =sess.run(accuracy, feed_dict ={X: batch_x, Y: batch_y})
             test = sess.run(accuracy, feed_dict ={X: x_input_test, Y: y_input_test})
             l_train = sess.run(loss, feed_dict = {X: batch_x, Y: batch_y})
             l_test = sess.run(loss, feed_dict ={X: x_input_test, Y: y_input_test}
                               
                               
    a.append(train)
    b.append(test)
    c.append(l_train)
    d.append(l_test)
    
    print(f'step: {step}')
    print('model training accuracy:')
    print(sess.run(accuracy, feed_dict ={X: batch_x, Y: batch_y}))                           
    print('model test accuracy:')      
    print(sess.run(accuracy, feed_dict ={X: x_input_test, Y: y_input_test}))                           
    
print('final model accuracy')
print(sess.run(accuracy, feed_dict ={X: x_input_test, Y: y_input_test}))  

In [ ]:
plt.plot(a, label='Train Accuracy')
plt.plot(b.linestyle='--', color='r', label='Test Accuracy')
plt.xlabel('Steps')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
plt.plot(c, label='Training Loss')
plt.plot(d.linestyle='--', color='r', label='Test Loss')
plt.xlabel('Steps')
plt.ylabel('Accuracy')
plt.legend()
plt.show()